In [9]:
import yaml
import logging
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor

import pandas as pd
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sns

from artool import ar_ana, ar_model


# set logging level
logging.basicConfig(level=logging.INFO)
logging.getLogger("artool").setLevel(logging.INFO)

In [2]:
# Config
model_config_path = Path("/home/yangzhe/Aura_study/model/fr_pred/1027_lgb/config.yaml")
with open(model_config_path) as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
model_dir = Path(cfg["model_dir"])

In [3]:
# Get inputs
print("Loading inputs...")
df = pd.read_feather(model_dir / "input.feather")
split_time = pd.to_datetime(cfg["train_test_split_date"])
df_tr, df_val, df_te = ar_model.train_utils.data_split_by_time(
    df, cfg["time"], split_time, val_ratio=0.2
)
t_tr = df_tr[cfg["time"]]
x_tr = df_tr[cfg["features"]]
y_tr = df_tr[cfg["target"]]
t_val = df_val[cfg["time"]]
x_val = df_val[cfg["features"]]
y_val = df_val[cfg["target"]]

t_te = df_te[cfg["time"]]
x_te = df_te[cfg["features"]]
y_te = df_te[cfg["target"]]
pr_te = df_te["price"]
fr_te = df_te["funding_rate"]


Loading inputs...


In [15]:
# Plot funding_rate__future_5 vs funding_rate__ewm2d

x = df_te["funding_rate__ewm2d"]
y = df_te["funding_rate__future_5"]
find_nan = pd.isna(x) | pd.isna(y)
x = x[~find_nan]
y = y[~find_nan]
r2 = r2_score(x, y)
corr = x.corr(y)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x, y, alpha=0.1)
ax.set_title(f"r2={r2:.3f}, corr={corr:.3f}")
ax.set_xlabel("funding_rate__ewm2d")
ax.set_ylabel("funding_rate__future_5")
fig.savefig("funding_rate__future_5_vs_funding_rate__ewm2d.png")

## Simu ewm

In [4]:
symbols = df_te["symbol"].unique()
trade_data_ewm = {}
for symbol in symbols:
    df_tmp = df_te[df_te["symbol"] == symbol]
    df_syb = pd.DataFrame()
    df_syb["time"] = df_tmp[cfg["time"]]
    df_syb["price"] = df_tmp["price"]
    df_syb["funding_rate"] = df_tmp["funding_rate"]
    df_syb["vol"] = df_tmp["vol"]
    df_syb["signal"] = df_tmp["funding_rate__ewm2d"]
    trade_data_ewm[symbol] = df_syb
ts_ewm = ar_ana.simu.TradeSimulatorSignalSimple(trade_data_ewm)

### Compare different capital sizes

In [5]:
def get_rec(cap):
    ts_ewm = ar_ana.simu.TradeSimulatorSignalSimple(trade_data_ewm)
    ts_ewm.trade(cap)
    plot_dir = Path(f"plot_ewm_cap_{int(cap / 1000)}k")
    plot_dir.mkdir(exist_ok=True)
    ts_ewm.plot_book(plot_dir)
    return ts_ewm

with ProcessPoolExecutor(max_workers=8) as executor:
    cap_list = [1e5, 1e6, 1e7]
    ts_ewm_list = executor.map(get_rec, cap_list)

INFO:artool:Simulating trading
  0%|          | 0/2208 [00:00<?, ?it/s]INFO:artool:Simulating trading
INFO:artool:Simulating trading
cum_pnl: 212598.41 / 10000000.00: 100%|██████████| 2208/2208 [04:39<00:00,  7.90it/s]


In [6]:
fig, ax = plt.subplots(figsize=(8, 6))
for ts in ts_ewm_list:
    label = f"cap: {int(ts.cap / 1000)}k"
    ts.plot_pnl_cum_curve_rate(ax=ax, label=label)
ax.legend()
plot_dir = Path(f"plot_ewm_compare")
plot_dir.mkdir(exist_ok=True)
fig.savefig(plot_dir / "pnl_cum_curve_rate.png")